In [1]:
import os
import datetime
import pandas as pd
import numpy as np
import glob

#表示行数の増加
pd.set_option('display.max_rows', 1500)

today = datetime.datetime.now()

y = today.year#today.year
m = today.month -1#today.month

ym = str(y) + str('{0:02d}'.format(m))
print(ym)
#サイト単位で集計するショップを指定
shopUnit = \
["TOKYO OUTLET WEEK ONLINE", "ファッション通販 blanccloset", "ENVYM", "ECLIN", "公式通販Fig＆Viper", "DURAS OFFICIAL WEB STORE",\
 "EGOIST(エゴイスト)ファッション公式通販サイト", "COOMB official online store", "TONAL", \
 "TRUNC 88(トランク エイティーエイト)ブランド公式サイト", "Brown&Street ONLINE", "CUBE SUGAR",\
 "Cuore ONLINE SHOP", "Yeti イエティ", "BUTSU/YOKU", "Port Of Call Online", "willfully ONLINE SHOP","AURALEE",\
 "STRAWBERRY-FIELDS OUTLET", "SHIBUYA109"]

#ブランド単位で集計するショップを指定
brandUnit = \
["STRIPE CLUB", "Arpege story", "CAN ONLINE SHOP", "CAN OUTLET", "Roomy's WEB STORE", "Ailand", "atomicboxx", "RABOKIGOSHI web store",\
 "SANKO BAZAAR", "URBAN RESEARCH OUTLET"]

#USED集計をするショップ名
usedUnit = \
["STRIPE CLUB", ]

#大カテゴリにて集計するショップ
category_L = \
["STRIPE CLUB", "Ailand", "ファッション通販 blanccloset", "ENVYM", "DURAS OFFICIAL WEB STORE", "URBAN RESEARCH OUTLET", \
 "EGOIST(エゴイスト)ファッション公式通販サイト", "atomicboxx", "SANKO BAZAAR","STRAWBERRY-FIELDS OUTLET"]

#中カテゴリにて集計するショップ
category_M = \
["ECLIN", "公式通販Fig＆Viper", "COOMB official online store", "TONAL","TRUNC 88(トランク エイティーエイト)ブランド公式サイト"]

#受注形態そのままにて集計するショップ
orderForm_default = \
["STRIPE CLUB", ]

#受注形態集計（PC + MB）にて集計するショップ
orderForm_sum = \
["CAN ONLINE SHOP", "CAN OUTLET"]

orderFormUnit = orderForm_default + orderForm_sum

#TOW受注金額
towOrderUnit = \
["ENVYM", "EGOIST(エゴイスト)ファッション公式通販サイト"]

#購入ステータス別にて集計するショップ
purchaseStatus = \
["TOKYO OUTLET WEEK ONLINE", "ファッション通販 blanccloset", "ENVYM", "ECLIN", "公式通販Fig＆Viper", "DURAS OFFICIAL WEB STORE",\
 "EGOIST(エゴイスト)ファッション公式通販サイト", "COOMB official online store", "TONAL", \
 "TRUNC 88(トランク エイティーエイト)ブランド公式サイト", "Brown&Street ONLINE", "STRAWBERRY-FIELDS OUTLET", "SHIBUYA109"] + \
["STRIPE CLUB", "KOE", "Arpege story", "CAN ONLINE SHOP", "CAN OUTLET", "Roomy's WEB STORE", "Ailand", \
 "atomicboxx", "RABOKIGOSHI web store", "URBAN RESEARCH OUTLET"]

#入庫集計対象のショップ（事業部指定）
receiveShop = \
['TOKYO OUTLET WEEK事業部', '英インターナショナル株式会社', 'ENVYM事業部', \
 'ECLIN事業部', 'FIG&VIPER', 'DURAS事業部', 'COOMB事業部', \
 'TRUNC 88事業部','109事業部']

receiveShop_convert = \
{'TOKYO OUTLET WEEK事業部':"TOKYO OUTLET WEEK ONLINE",\
 '英インターナショナル株式会社':"ファッション通販 blanccloset", \
 'ENVYM事業部':"ENVYM", \
 'ECLIN事業部':"ECLIN",\
 'FIG&VIPER':"公式通販Fig＆Viper",\
 'DURAS事業部':"DURAS OFFICIAL WEB STORE", 
 'COOMB事業部':"COOMB official online store", \
 'TRUNC 88事業部':"TRUNC 88(トランク エイティーエイト)ブランド公式サイト",\
 '109事業部':"SHIBUYA109"}

#入庫集計対象のショップ（ブランド単位）
receiveBrand = \
['STRIPE CLUB事業部', 'Arpege story事業部', "Roomy's WEB STORE事業部"]

receiveBrand_convert = \
{'STRIPE CLUB事業部':"STRIPE CLUB",\
 'Arpege story事業部':"Arpege story",\
 "Roomy's WEB STORE事業部":"Roomy's WEB STORE", \
 "Ailand":"Ailand"}

regMember_convert = \
{"EGOIST":"EGOIST(エゴイスト)ファッション公式通販サイト", \
 "TOKYO OUTLET WEEK":"TOKYO OUTLET WEEK ONLINE",\
 "Brown&Street":"Brown&Street ONLINE",\
 'RABOKIGOSHI公式通販会員':"RABOKIGOSHI web store"}

regMemberTotalUnit = \
["Arpege story", "TOKYO OUTLET WEEK ONLINE", "CAN ONLINE SHOP", "CAN OUTLET", "Roomy's WEB STORE",
 "RABOKIGOSHI web store", "DURAS OFFICIAL WEB STORE", "EGOIST(エゴイスト)ファッション公式通販サイト", "COOMB official online store",\
 "SHIBUYA109","STRAWBERRY-FIELDS OUTLET"]

#暫定でのroot dir
root = "/Users/miyamoto/Documents/Root/raw/monthly_agg"

202101


In [2]:
#共通関数群

def file_ditect(key_dir):#指定ディレクトリに存在するファイルパスの一覧取得
    path = root + "/" + key_dir + "/" + ym
    
    files = []

    # パスで指定したファイルの一覧をリスト形式で取得
    files = glob.glob(path + '/*.csv')

    #print(files)

    for file in files:
        print(file)
    
    return files
        
def stripe_load(st):#ショップ名"KOE"を"STRIPE CLUB"に、ブランド名を"Melan Cleuge"に変換
    #st = pd.read_csv(file)
    st = st.replace({"KOE" : "STRIPE CLUB"})
    st = st.replace({"Melan Cleuge Men's":"Melan Cleuge", "Melan Cleuge Women's":"Melan Cleuge"})
    
    return st

def zero_load(ze):
    #ze = pd.read_csv(file)
    ze = ze.query("ショップ名 != 'PALEMOBA ONLINESTORE'")
    
    return ze

def import_csv(df, file):
    if "stripe" in file:
        df = stripe_load(df)
    elif "zero" in file:
        df = zero_load(df)
    #else:
    #   df = pd.read_csv(file)

    df = df[~df["ブランド名"].str.contains("ノベルティ")]
    
    return df

In [3]:
#sell用の関数群

def import_sell(file):
    dataframe = pd.read_csv(file)
    dataframe["年"] = pd.to_datetime(dataframe["日付"]).dt.year
    dataframe["月"] = pd.to_datetime(dataframe["日付"]).dt.month
    dataframe["日"] = pd.to_datetime(dataframe["日付"]).dt.day
    df = import_csv(dataframe, file)
    
    return df

def shop_sell(df):
    agg = df.query("ショップ名 in @shopUnit")
    
    if agg.empty:
        return
    
    agg = agg.groupby(["年", "月", "ショップ名"]).agg({'出荷金額':'sum','返品金額':'sum','出荷金額-返品金額':'sum'})

    agg.rename(columns={'出荷金額-返品金額': '売上金額'}, inplace=True)
    
    agg = agg.stack()
    
    return agg

def used_sell(df):
    used = df.query('ショップ名 in @usedUnit')
    
    if used.empty:
        return
    
    used = used.query('商品種別 == "USED"')
    
    used_agg = used.groupby(["年", "月", "ショップ名", "ブランド名"]).agg({'出荷金額-返品金額':'sum'})

    used_agg.rename(columns={'出荷金額-返品金額': 'USED売上金額'}, inplace=True)
    
    used_agg = used_agg.stack().reset_index()

    return used_agg

def brand_sell(df):
    agg_brand = df.query("ショップ名 in @brandUnit")
    
    if agg_brand.empty:
        return

    agg_brand = agg_brand.groupby(["年", "月", "ショップ名", "ブランド名"]).agg({'出荷金額':'sum','返品金額':'sum','出荷金額-返品金額':'sum'})

    agg_brand.rename(columns={'出荷金額-返品金額': '売上金額'}, inplace=True)
    
    agg_brand = agg_brand.stack().reset_index()

    return agg_brand

In [4]:
#sell処理
files_sell = file_ditect("sell")
sell_list_shop = []
sell_list_brand = []
sell_list_subbrand = []

for file in files_sell:
    sell = import_sell(file)
    df_shop_sell = shop_sell(sell)
    df_brand_sell = brand_sell(sell)
    df_used_sell = used_sell(sell)
    sell_list_shop.append(df_shop_sell)
    sell_list_brand.extend([df_brand_sell, df_used_sell])

/Users/miyamoto/Documents/Root/raw/monthly_agg/sell/202101/売上（出荷-顧客返品）_20210202080825_stripe.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/sell/202101/売上（出荷-顧客返品）_20210202080953_CAN.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/sell/202101/売上（出荷-顧客返品）_20210202081127_AP.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/sell/202101/売上（出荷-顧客返品）_20210202080715_new.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/sell/202101/売上（出荷-顧客返品）_20210202081203_TOW.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/sell/202101/売上（出荷-顧客返品）_20210202080917_zero.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/sell/202101/売上（出荷-顧客返品）_20210202081052_shibuya.csv


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (11,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
sh_sell = pd.concat(sell_list_shop,axis = 0).reset_index()
sh_sell = sh_sell[sh_sell.columns[sh_sell.columns != 'index']]
sh_sell.rename(columns={"level_3":"属性", 0:"scalar"}, inplace =True)

br_sell = pd.concat(sell_list_brand,axis = 0).reset_index()
br_sell = br_sell[br_sell.columns[br_sell.columns != 'index']]
br_sell.rename(columns={"level_4":"属性", 0:"scalar"}, inplace =True)

In [6]:
sh_sell

,年,月,ショップ名,属性,scalar
0,2021,1,ECLIN,出荷金額,1650789
1,2021,1,ECLIN,返品金額,74996
2,2021,1,ECLIN,売上金額,1575793
3,2021,1,ENVYM,出荷金額,3076340
4,2021,1,ENVYM,返品金額,399600
5,2021,1,ENVYM,売上金額,2676740
6,2021,1,公式通販Fig＆Viper,出荷金額,1903452
7,2021,1,公式通販Fig＆Viper,返品金額,139000
8,2021,1,公式通販Fig＆Viper,売上金額,1764452
9,2021,1,TOKYO OUTLET WEEK ONLINE,出荷金額,131058


In [7]:
#//sell処理

In [8]:
#order用の関数群
def import_order(file):
    df = pd.read_csv(file)    
    
    shopList = df["ショップ名"].unique()
    #アウトレットはそのまま出すショップ名
    outlet_sellStatus =["TOKYO OUTLET WEEK ONLINE", "STRIPE CLUB", "CAN ONLINE SHOP", "CAN OUTLET","atomicboxx"]
    #全てそのまま出すショップ名
    doNothing_sellStatus = ["STRAWBERRY-FIELDS OUTLET", "SHIBUYA109"]
    
    for shop in shopList:
        if shop in doNothing_sellStatus:#109は販売ステータス全てそのまま
            continue
        elif shop in outlet_sellStatus:
            df_query = "ショップ名 == @shop"
            target = df.query(df_query)
            df.loc[target.index, "販売ステータス"] = df.loc[target.index, "販売ステータス"].replace({"セール（シークレット）":"セール", '予約（シークレット）':"予約"})
        else:
            df_query = "ショップ名 == @shop"
            target = df.query(df_query)
            df.loc[target.index, "販売ステータス"] = df.loc[target.index, "販売ステータス"].replace({"セール（シークレット）":"セール", 'アウトレット':"セール", '予約（シークレット）':"予約"})
    
    df["年"] = pd.to_datetime(df["受注日"]).dt.year
    df["月"] = pd.to_datetime(df["受注日"]).dt.month
    df["日"] = pd.to_datetime(df["受注日"]).dt.day
    df["大カテゴリ名"] = "受注金額/大カテゴリ/" + df["大カテゴリ名"]
    df["中カテゴリ名"] = "受注金額/中カテゴリ/" + df["中カテゴリ名"]
    df["販売ステータス"] = "受注金額/販売ステータス/" + df["販売ステータス"]
    df["購入ステータス"] = "受注件数/購入ステータス/" + df["購入ステータス"]
    
    df = import_csv(df, file)
    
    return df

def shop_order(df):
    df = df.query("ショップ名 in @shopUnit")
    
    if df.empty:
        return
    
    agg = df.query("受注金額 != 0")
    agg = df.groupby(["年", "月", "ショップ名"]).agg({'受注番号':'nunique','受注数':'sum', '受注金額':'sum'})
    agg.rename(columns={'受注番号': '受注件数'}, inplace=True)
    agg = agg.stack()
    
    agg_sellStatus = df.groupby(["年", "月", "ショップ名", "販売ステータス"]).agg({'受注金額':'sum'})
    agg_sellStatus = agg_sellStatus.stack()
    
    agg_category_L = df.groupby(["年", "月", "ショップ名", "大カテゴリ名"]).agg({'受注金額':'sum'})
    agg_category_L = agg_category_L.query("ショップ名 in @category_L")
    agg_category_L = agg_category_L.stack()
    
    agg_category_M = df.groupby(["年", "月", "ショップ名", "中カテゴリ名"]).agg({'受注金額':'sum'})
    agg_category_M = agg_category_M.query("ショップ名 in @category_M")
    agg_category_M = agg_category_M.stack()
    
    agg_purStatus = df.query(("ショップ名 in @purchaseStatus & 受注金額 != 0"))
    agg_purStatus = agg_purStatus.groupby(["年", "月", "ショップ名", "購入ステータス"]).agg({'受注番号':'nunique'})
    agg_purStatus.rename(columns={'受注番号': '受注件数'}, inplace=True)
    agg_purStatus = agg_purStatus.stack()
    
    df_agg = pd.concat([agg, agg_sellStatus, agg_category_L, agg_category_M, agg_purStatus], axis = 0) #agg_orderForm, 
    
    return df_agg
    
def used_order(df):
    used = df.query('ショップ名 in @usedUnit')
    
    if used.empty:
        return
    
    used = used.query('商品種別 == "USED"')

    used_agg = used.groupby(["年", "月", "ショップ名", "ブランド名"]).agg({'受注金額':'sum'})
    
    used_agg.rename(columns={'受注金額': 'USED受注金額'}, inplace=True)
    
    used_agg = used_agg.stack().reset_index()
    
    used_agg.rename(columns={'level_4': '属性'}, inplace=True)

    return used_agg

def orderForm_order(df):
    orderForm = df.query('ショップ名 in @orderFormUnit')
    
    if orderForm.empty:
        return
    
    agg_orderForm = orderForm.query("受注金額 != 0")
    
    '''agg_orderForm_def = orderForm.query("ショップ名 in @orderForm_default")
    agg_orderForm_sum = orderForm.query("ショップ名 in @orderForm_sum")
    agg_orderForm_sum = agg_orderForm_sum.replace({"PC":"PC + MB", 'MB':"PC + MB"})
     
    agg_orderForm = pd.concat([agg_orderForm_def, agg_orderForm_sum], axis = 0, sort = True)'''
    
    agg_orderForm["受注形態"] = "受注形態/" + agg_orderForm["受注形態"]
    agg_orderForm = agg_orderForm.groupby(["年", "月", "ショップ名", "受注形態"]).agg({'受注番号':'nunique'})
    agg_orderForm.rename(columns={'受注番号': '受注件数'}, inplace=True)
    agg_orderForm = agg_orderForm.stack().reset_index()
    
    agg_orderForm["属性"] = agg_orderForm["受注形態"]
    
    return agg_orderForm

def TOW_order(df):
    tow = df.query('ショップ名 == "TOKYO OUTLET WEEK ONLINE" ')
    
    if tow.empty:
        return
    
    agg = tow.query("ブランド名 in @ towOrderUnit")
    
    if agg.empty:
        return
    
    agg = df.groupby(["年", "月", "ブランド名", "サブブランド名"]).agg({'受注金額':'sum'})
    agg.rename(columns={'ブランド名': 'ショップ名', 'サブブランド名': 'ブランド名', '受注金額':'TOW受注金額'}, inplace=True)
    agg = agg.stack()
    
    return agg

def brand_order(df):
    df = df.query("ショップ名 in @brandUnit")
    
    if df.empty:
        return
    
    agg = df.query("受注金額 != 0")
    agg = agg.groupby(["年", "月", "ショップ名", "ブランド名"]).agg({'受注金額':'sum', '受注番号':'nunique','受注数':'sum'})
    agg.rename(columns={'受注番号': '受注件数'}, inplace=True)
    agg = agg.stack()
    
    agg_site = df.query("受注金額 != 0")
    agg_site = agg_site.groupby(["年", "月", "ショップ名"]).agg({'受注番号':'nunique','受注数':'sum'})
    agg_site.rename(columns={'受注番号': '受注件数'}, inplace=True)
    agg_site = agg_site.stack()
    
    agg_sellStatus = df.groupby(["年", "月", "ショップ名", "ブランド名", "販売ステータス"]).agg({'受注金額':'sum'})
    agg_sellStatus = agg_sellStatus.stack()
    
    agg_category_L = df.query("ショップ名 in @category_L")
    agg_category_L = agg_category_L.groupby(["年", "月", "ショップ名", "ブランド名", "大カテゴリ名"]).agg({'受注金額':'sum'})
    agg_category_L = agg_category_L.stack()
    
    agg_category_Arpege = df.query("ショップ名 == 'Arpege story' ")
    agg_category_Arpege["カテゴリコード"] = agg_category_Arpege["メーカー品番"].astype(str).str[3] #4文字目抽出
    agg_category_Arpege = pd.merge(agg_category_Arpege, arpege_category, on=["カテゴリコード"])
    agg_category_Arpege = agg_category_Arpege.groupby(["年", "月", "ショップ名", "ブランド名", "カテゴリ名"]).agg({'受注金額':'sum'})
    agg_category_Arpege = agg_category_Arpege.stack()
    
    roomy = "Roomy's WEB STORE"
    agg_category_roomy = df.query('ショップ名 == @roomy ')
    agg_category_roomy["カテゴリコード"] = agg_category_roomy["メーカー品番"].astype(str).str[-1] #最後の1文字目抽出
    agg_category_roomy = pd.merge(agg_category_roomy, roomy_category, on=["カテゴリコード"])
    agg_category_roomy = agg_category_roomy.groupby(["年", "月", "ショップ名", "ブランド名", "カテゴリ名"]).agg({'受注金額':'sum'})
    agg_category_roomy = agg_category_roomy.stack()
    
    agg_purStatus = df.query(("ショップ名 in @purchaseStatus & 受注金額 != 0"))
    agg_purStatus = agg_purStatus.groupby(["年", "月", "ショップ名", "購入ステータス"]).agg({'受注番号':'nunique'})
    agg_purStatus.rename(columns={'受注番号': '受注件数'}, inplace=True)
    agg_purStatus = agg_purStatus.stack()
    
    df_agg = pd.concat([agg,  agg_sellStatus, agg_category_L, agg_category_Arpege, agg_category_roomy], axis = 0) #agg_orderForm, 
    df_agg = df_agg.reset_index()
    df_agg.rename(columns={'level_4': '属性'}, inplace=True)
    
    df_agg_shop = pd.concat([agg_site,  agg_purStatus], axis = 0)
    df_agg_shop = df_agg_shop.reset_index()
    df_agg_shop.rename(columns={'level_3': '属性'}, inplace=True)

    agg_total = pd.concat([df_agg, df_agg_shop], axis = 0)
    
    return agg_total

def shop_orderForm(df, file):
    df = df.query("ショップ名 in @shopUnit")
    
    if df.empty:
        return
    
    df["受注形態"] = "受注形態/" + df["受注形態"]
    df = df.groupby(["年", "月", "ショップ名", "受注形態"]).agg({'受注件数':'sum'})
    df = df.stack()
    
    return df

def brand_orderForm(df, file):
    df = df.query("ショップ名 in @brandUnit")
    
    if df.empty:
        return
    
    df["受注形態"] = "受注形態/" + df["受注形態"]
    df = df.groupby(["年", "月", "ショップ名", "受注形態"]).agg({'受注件数':'sum'})
    df = df.stack().reset_index()
    df["属性"] = df["受注形態"]
    return df
    

In [9]:
#order処理
files_order = file_ditect("order")
files_order_orderForm = file_ditect("order/" + ym + "/orderForm")
ap_cateFile = glob.glob(root + "/order"  + '/Arpege*.xlsx')[0]
arpege_category = pd.read_excel(ap_cateFile, dtype = str)
ro_cateFile = glob.glob(root + "/order"  + '/Roomy*.xlsx')[0]
roomy_category = pd.read_excel(ro_cateFile,  dtype = str)

order_list_shop = []
order_list_brand = []

for file in files_order:
    order = import_order(file)

    df_shop_order = shop_order(order)
    df_brand_order = brand_order(order)
    df_orderForm_order = orderForm_order(order)
    df_TOW_order = TOW_order(order)
    df_used_order = used_order(order)

    order_list_shop.extend([df_shop_order, df_TOW_order])
    order_list_brand.extend([df_brand_order, df_used_order, df_orderForm_order])
    
for file in files_order_orderForm:
    orderForm = pd.read_csv(file)
    
    df_shop_orderForm = shop_orderForm(orderForm, file)
    df_brand_orderForm = brand_orderForm(orderForm, file)
    order_list_shop.append(df_shop_orderForm)
    order_list_brand.append(df_brand_orderForm)

/Users/miyamoto/Documents/Root/raw/monthly_agg/order/202101/受注_20210201231656_CAN.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/order/202101/受注_20210201231848_zero.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/order/202101/受注_20210201231326_TOW.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/order/202101/受注_20210202021503_new.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/order/202101/受注_20210201231438_AP.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/order/202101/受注_20210201231602_shibuya.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/order/202101/受注_20210201232037_stripe.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/order/202101/orderForm/202101/受注_20210201231255_TOW.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/order/202101/orderForm/202101/受注_20210202021325_new.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/order/202101/orderForm/202101/受注_20210201231528_shibuya.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/order/202101/orderForm/202101/受

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:173: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py

In [10]:
sh_order = pd.concat(order_list_shop,axis = 0).reset_index()
sh_order = sh_order[sh_order.columns[sh_order.columns != 'index']]
sh_order.rename(columns={"level_3":"属性", 0:"scalar"}, inplace =True)

br_order = pd.concat(order_list_brand,axis = 0).reset_index()
br_order = br_order[br_order.columns[br_order.columns != 'index']]
br_order.drop("level_4", axis=1)
br_order.rename(columns={0:"scalar"}, inplace =True)
br_order = br_order[["年", "月", "ショップ名", "ブランド名", "属性", "scalar"]]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [11]:
sh_order

,年,月,ショップ名,属性,scalar
0,2021,1,AURALEE,受注件数,300
1,2021,1,AURALEE,受注数,398
2,2021,1,AURALEE,受注金額,11058700
3,2021,1,BUTSU/YOKU,受注件数,9
4,2021,1,BUTSU/YOKU,受注数,9
5,2021,1,BUTSU/YOKU,受注金額,154700
6,2021,1,Brown&Street ONLINE,受注件数,334
7,2021,1,Brown&Street ONLINE,受注数,705
8,2021,1,Brown&Street ONLINE,受注金額,4673960
9,2021,1,COOMB official online store,受注件数,166


In [12]:
#//order処理

In [13]:
#newProduct関数群
def import_newProduct(file):
    dataframe = pd.read_csv(file)
    df = import_csv(dataframe, file)
    return df

def shop_newProduct(df):
    shopUnit_ex = shopUnit + ["Ailand", "URBAN RESEARCH OUTLET"]
    df = df.query("ショップ名 in @shopUnit_ex")
    
    if df.empty:
        return
    
    agg = df.groupby(["年", "月", "ショップ名"]).agg({'品番数':'sum','日':'nunique'})
    agg.rename(columns={'品番数':'新規アップ数', '日': 'アップ日数'}, inplace=True)
    agg = agg.stack()
    
    return agg

def brand_newProduct(df):
    brandUnit_ex = brandUnit.copy()
    for i in ["Ailand", "URBAN RESEARCH OUTLET", "STRIPE CLUB", "CAN ONLINE SHOP", "CAN OUTLET"]:
        brandUnit_ex.remove(i)
    df = df.query("ショップ名 in @brandUnit_ex")
    
    if df.empty:
        return
    
    agg = df.groupby(["年", "月",  "ショップ名", "ブランド名"]).agg({'品番数':'sum','日':'nunique', "SKU数":"sum"})
    agg.rename(columns={'品番数':'新規アップ数', '日': 'アップ日数'}, inplace=True)
    agg = agg.stack()

    return agg

In [14]:
#newProduct処理
files_newProduct = file_ditect("newProduct")

newProduct_list_shop = []
newProduct_list_brand = []

for file in files_newProduct:
    newProduct = import_newProduct(file)

    df_shop_newProduct = shop_newProduct(newProduct)
    df_brand_newProduct = brand_newProduct(newProduct)

    newProduct_list_shop.append(df_shop_newProduct)
    newProduct_list_brand.append(df_brand_newProduct)

/Users/miyamoto/Documents/Root/raw/monthly_agg/newProduct/202101/新規商品アップ数_20210201223836_AP.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/newProduct/202101/新規商品アップ数_20210201223704_new.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/newProduct/202101/新規商品アップ数_20210201223911_TOW.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/newProduct/202101/新規商品アップ数_20210201223801_zero.csv


In [15]:
sh_newProduct = pd.concat(newProduct_list_shop,axis = 0).reset_index()
sh_newProduct = sh_newProduct[sh_newProduct.columns[sh_newProduct.columns != 'index']]
sh_newProduct.rename(columns={"level_3":"属性", 0:"scalar"}, inplace =True)

br_newProduct = pd.concat(newProduct_list_brand,axis = 0).reset_index()
br_newProduct = br_newProduct[br_newProduct.columns[br_newProduct.columns != 'index']]
br_newProduct.rename(columns={"level_4":"属性", 0:"scalar"}, inplace =True)

In [16]:
sh_newProduct

,年,月,ショップ名,属性,scalar
0,2021,1,Ailand,新規アップ数,161
1,2021,1,Ailand,アップ日数,15
2,2021,1,ENVYM,新規アップ数,27
3,2021,1,ENVYM,アップ日数,3
4,2021,1,公式通販Fig＆Viper,新規アップ数,9
5,2021,1,公式通販Fig＆Viper,アップ日数,5
6,2021,1,TOKYO OUTLET WEEK ONLINE,新規アップ数,1324
7,2021,1,TOKYO OUTLET WEEK ONLINE,アップ日数,25
8,2021,1,AURALEE,新規アップ数,32
9,2021,1,AURALEE,アップ日数,2


In [17]:
#request関数群
def import_request(file):
    df = pd.read_csv(file)
    
    df["リクエスト金額"] = df["定価"]*df["登録数"]
    
    df["年"] = pd.to_datetime(df["対象日"]).dt.year
    df["月"] = pd.to_datetime(df["対象日"]).dt.month
    df["日"] = pd.to_datetime(df["対象日"]).dt.day
    
    df = df.query("リクエストタイプ == '再入荷' ")
    
    df = import_csv(df, file)
    
    return df

def shop_request(df):
    shopUnit_ex = shopUnit + ["Ailand", "URBAN RESEARCH OUTLET"]
    df = df.query("ショップ名 in @shopUnit_ex")
    
    if df.empty:
        return
    
    df_reg = df.query("登録数 != 0")
    df_send = df.query("送信数 != 0")
    
    agg_reg = df_reg.groupby(["年", "月", "ショップ名"]).agg({'商品コード':'nunique','リクエスト金額':'sum'})
    agg_reg.rename(columns={'商品コード': '登録品番数'}, inplace=True)
    
    agg_send = df_send.groupby(["年", "月", "ショップ名"]).agg({'商品コード':'nunique'})
    agg_send.rename(columns={'商品コード': '送信品番数'}, inplace=True)
    
    agg = pd.merge(agg_reg, agg_send, on=["年", "月", "ショップ名"], how='left')
    
    agg = agg.stack()
    
    return agg

def brand_request(df):
    bU = brandUnit.copy()
    for i in ["Ailand", "URBAN RESEARCH OUTLET"]:
        bU.remove(i)
    df = df.query("ショップ名 in @bU")
    
    if df.empty:
        return
    
    df_stripe = df.query("ショップ名 == 'STRIPE CLUB' ")
    df = df.query("ショップ名 != 'STRIPE CLUB' ")
    
    if df_stripe.empty:
        
        df_reg = df.query("登録数 != 0")
        df_send = df.query("送信数 != 0")

        agg_reg = df_reg.groupby(["年", "月", "ショップ名", "ブランド名"]).agg({'商品コード':'nunique','リクエスト金額':'sum'})
        agg_reg.rename(columns={'商品コード': '登録品番数'}, inplace=True)

        agg_send = df_send.groupby(["年", "月", "ショップ名", "ブランド名"]).agg({'商品コード':'nunique'})
        agg_send.rename(columns={'商品コード': '送信品番数'}, inplace=True)

        agg = pd.merge(agg_reg, agg_send, on=["年", "月", "ショップ名", "ブランド名"], how='left')

        agg = agg.stack()

        return agg
    
    else:#stripe例外
        df_stripe["販売単価"] = \
        df_stripe["販売単価"].where(df_stripe["販売単価"] != '< NULL >',df_stripe["定価"]).astype("float64")

        df_reg_stripe = df_stripe.query("登録数 != 0")
        df_send_stripe = df_stripe.query("送信数 != 0")

        df_reg_stripe["リクエスト金額_定価"] = df_reg_stripe["登録数"] * df_reg_stripe["定価"]
        df_reg_stripe["リクエスト金額_販売単価"] = df_reg_stripe["登録数"] * df_reg_stripe["販売単価"]

        agg_reg_stripe = df_reg_stripe.groupby(["年", "月", "ショップ名", "ブランド名"]).agg({'商品コード':'nunique','リクエスト金額_定価':'sum', 'リクエスト金額_販売単価':'sum'})
        agg_reg_stripe.rename(columns={'商品コード': '登録品番数'}, inplace=True)

        agg_send_stripe = df_send_stripe.groupby(["年", "月", "ショップ名", "ブランド名"]).agg({'商品コード':'nunique'})
        agg_send_stripe.rename(columns={'商品コード': '送信品番数'}, inplace=True)

        agg_stripe = pd.merge(agg_reg_stripe, agg_send_stripe, on=["年", "月", "ショップ名", "ブランド名"], how='left')
        
        agg_stripe = agg_stripe.stack()

        return agg_stripe

In [18]:
#request処理
files_request = file_ditect("request")

/Users/miyamoto/Documents/Root/raw/monthly_agg/request/202101/リクエストメール集計_20210202022816_TOW.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/request/202101/リクエストメール集計_20210202022844_AP.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/request/202101/リクエストメール集計_20210202080455_new.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/request/202101/リクエストメール集計_20210202022916_CAN.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/request/202101/リクエストメール集計_20210202080420_stripe.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/request/202101/リクエストメール集計_20210202022947_zero.csv


In [19]:
request_list_shop = []
request_list_brand = []

for file in files_request:
    request = import_request(file)

    df_shop_request = shop_request(request)
    df_brand_request = brand_request(request)

    request_list_shop.append(df_shop_request)
    request_list_brand.append(df_brand_request)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
sh_request = pd.concat(request_list_shop,axis = 0).reset_index()
sh_request = sh_request[sh_request.columns[sh_request.columns != 'index']]
sh_request.rename(columns={"level_3":"属性", 0:"scalar"}, inplace =True)

br_request = pd.concat(request_list_brand,axis = 0).reset_index()
br_request = br_request[br_request.columns[br_request.columns != 'index']]
br_request.rename(columns={"level_4":"属性", 0:"scalar"}, inplace =True)

In [21]:
sh_request

,年,月,ショップ名,属性,scalar
0,2021,1,TOKYO OUTLET WEEK ONLINE,登録品番数,88.0
1,2021,1,TOKYO OUTLET WEEK ONLINE,リクエスト金額,1078956.0
2,2021,1,TOKYO OUTLET WEEK ONLINE,送信品番数,36.0
3,2021,1,Ailand,登録品番数,698.0
4,2021,1,Ailand,リクエスト金額,27046992.0
5,2021,1,Ailand,送信品番数,225.0
6,2021,1,ECLIN,登録品番数,62.0
7,2021,1,ECLIN,リクエスト金額,1540000.0
8,2021,1,ECLIN,送信品番数,25.0
9,2021,1,ENVYM,登録品番数,102.0


In [22]:
#receive関数群
def import_receive(file):
    df = pd.read_csv(file)
    
    df["年"] = pd.to_datetime(df["入庫実施日"]).dt.year
    df["月"] = pd.to_datetime(df["入庫実施日"]).dt.month
    df["日"] = pd.to_datetime(df["入庫実施日"]).dt.day
    
    #df = import_csv(df, file)
    
    if "stripe" in file:
        df = stripe_load(df)
    
    if "stripe" in file:
        df["入庫実績"] = df["定価"] * df["入庫実績数"]
    
    df = df[~df["ブランド名"].str.contains("ノベルティ")]
    
    return df

def shop_receive(df):
    df = df.query("事業部 in @receiveShop")
    df = df.replace(receiveShop_convert)
    
    if df.empty:
        return
    
    shopList = df["事業部"].unique()
    stockSettingUnit = ["ENVYM", "ECLIN", "DURAS OFFICIAL WEB STORE"] #在庫設定単位で出力するサイト
    concatenate = []
    
    for shop in shopList:
        df_for = df.query("事業部 == @shop")
        if shop in stockSettingUnit:
            agg_stockSet = df_for.groupby(["年", "月", "事業部", "在庫設定"]).agg({'入庫実績':'sum'})
            agg_stockSet.rename(columns={'入庫実績':'入庫金額'}, inplace=True)
            agg_stockSet = agg_stockSet.stack().reset_index()
            agg_stockSet["属性"] = agg_stockSet["level_4"] + "/" + agg_stockSet["在庫設定"]
            
            agg_other = df_for.groupby(["年", "月", "事業部"]).agg({'入庫実績数':'sum', '商品コード':'nunique'})
            agg_other.rename(columns={'入庫実績数':'入庫点数' , '商品コード': '入庫型数'}, inplace=True)
            agg_other = agg_other.stack().reset_index()
            agg_other.rename(columns={'level_3':'属性'}, inplace=True)
            concatenate.extend([agg_stockSet, agg_other])
        else:
            agg = df_for.groupby(["年", "月", "事業部"]).agg({'入庫実績':'sum','入庫実績数':'sum', '商品コード':'nunique'})
            agg.rename(columns={'入庫実績':'入庫金額','入庫実績数':'入庫点数' , '商品コード': '入庫型数'}, inplace=True)
            agg = agg.stack().reset_index()
            agg.rename(columns={'level_3':'属性'}, inplace=True)
            concatenate.append(agg)
    
    agg_return = pd.concat(concatenate, axis = 0)
    
    return agg_return

def brand_receive(df):
    df = df.query("事業部 in @receiveBrand")
    df = df.replace(receiveBrand_convert)
    print(df)
    
    if df.empty:
        return
    
    agg = df.groupby(["年", "月", "事業部", "ブランド名"]).agg({'入庫実績':'sum','入庫実績数':'sum', '商品コード':'nunique'})
    agg.rename(columns={'入庫実績':'入庫金額','入庫実績数':'入庫点数' , '商品コード': '入庫型数'}, inplace=True)
    
    agg = agg.stack()

    return agg

In [23]:
#receive処理
files_receive = file_ditect("receive")

/Users/miyamoto/Documents/Root/raw/monthly_agg/receive/202101/入庫実績_20210202021826_stripe.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/receive/202101/入庫実績_20210202021622_TOW.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/receive/202101/入庫実績_20210202021652_AP.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/receive/202101/入庫実績_20210202021902_new.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/receive/202101/入庫実績_20210202021754_zero.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/receive/202101/入庫実績_20210202021727_shibuya.csv


In [24]:
receive_list_shop = []
receive_list_brand = []

for file in files_receive:
    print(file)
    receive = import_receive(file)

    df_shop_receive = shop_receive(receive)
    df_brand_receive = brand_receive(receive)

    receive_list_shop.append(df_shop_receive)
    receive_list_brand.append(df_brand_receive)

/Users/miyamoto/Documents/Root/raw/monthly_agg/receive/202101/入庫実績_20210202021826_stripe.csv
       入庫予定No    発送元（取引先）          事業部     入庫区分       入庫予定日       入庫実施日  在庫反映  \
0      959958  マガシーク㈱（在庫）  STRIPE CLUB  預かり（委託）  2020/12/10  2021/01/07  反映済み   
1      959958  マガシーク㈱（在庫）  STRIPE CLUB  預かり（委託）  2020/12/10  2021/01/07  反映済み   
2      959958  マガシーク㈱（在庫）  STRIPE CLUB  預かり（委託）  2020/12/10  2021/01/07  反映済み   
3      959958  マガシーク㈱（在庫）  STRIPE CLUB  預かり（委託）  2020/12/10  2021/01/07  反映済み   
4      959958  マガシーク㈱（在庫）  STRIPE CLUB  預かり（委託）  2020/12/10  2021/01/07  反映済み   
...       ...         ...          ...      ...         ...         ...   ...   
44647  986801    ZOZO（在庫）  STRIPE CLUB    倉庫間移動  2021/01/31  2021/01/31  反映済み   
44648  986801    ZOZO（在庫）  STRIPE CLUB    倉庫間移動  2021/01/31  2021/01/31  反映済み   
44649  986801    ZOZO（在庫）  STRIPE CLUB    倉庫間移動  2021/01/31  2021/01/31  反映済み   
44650  986801    ZOZO（在庫）  STRIPE CLUB    倉庫間移動  2021/01/31  2021/01/31  反映済み   
44651  986801   

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [25]:
sh_receive = pd.concat(receive_list_shop,axis = 0).reset_index()
sh_receive = sh_receive[sh_receive.columns[sh_receive.columns != 'index']]
sh_receive.rename(columns={"level_3":"属性", 0:"scalar","事業部":"ショップ名"}, inplace =True)
sh_receive = sh_receive[["年", "月", "ショップ名", "属性", "scalar"]]

br_receive = pd.concat(receive_list_brand,axis = 0).reset_index()
br_receive = br_receive[br_receive.columns[br_receive.columns != 'index']]
br_receive.rename(columns={"level_4":"属性", 0:"scalar","事業部":"ショップ名"}, inplace =True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [26]:
sh_receive

,年,月,ショップ名,属性,scalar
0,2021,1,TOKYO OUTLET WEEK ONLINE,入庫金額,281870
1,2021,1,TOKYO OUTLET WEEK ONLINE,入庫点数,25
2,2021,1,TOKYO OUTLET WEEK ONLINE,入庫型数,24
3,2021,1,ENVYM,入庫金額/在庫連動,5555600
4,2021,1,ENVYM,入庫点数,989
5,2021,1,ENVYM,入庫型数,121
6,2021,1,公式通販Fig＆Viper,入庫金額,1759800
7,2021,1,公式通販Fig＆Viper,入庫点数,256
8,2021,1,公式通販Fig＆Viper,入庫型数,50
9,2021,1,ECLIN,入庫金額/在庫連動,1108500


In [27]:
#regMember関数群
def import_regMember(file):
    df = pd.read_csv(file)
    
    df["年"] = pd.to_datetime(df["申込日"]).dt.year
    df["月"] = pd.to_datetime(df["申込日"]).dt.month
    df["日"] = pd.to_datetime(df["申込日"]).dt.day
    
    if "stripe" in file:
        df.rename(columns={'登録サイト':'会員グループ'}, inplace=True)
    
    if "CAN" in file:
        df.rename(columns={'登録サイト':'会員グループ'}, inplace=True)
    
    #df = import_csv(df, file)
    
    df.rename(columns={'会員グループ':'ショップ名'}, inplace=True)
    
    df = df.replace(regMember_convert)
    
    #df = df.query("初回受注日 != '< NULL >' | ショップ名 != 'atomicboxx'")
    
    return df

def import_regMember_total(file):
    df = pd.read_csv(file)
    
    df["年"] = y
    df["月"] = m
    
    if "stripe" in file:
        df.rename(columns={'登録サイト':'会員グループ'}, inplace=True)
    
    if "CAN" in file:
        df.rename(columns={'登録サイト':'会員グループ'}, inplace=True)
        
    df.rename(columns={'会員グループ':'ショップ名'}, inplace=True)
    
    return df

def shop_regMember(df):
    shopUnit_ex = shopUnit.copy()
    for i in ["CUBE SUGAR", "Cuore ONLINE SHOP", "Yeti イエティ", "BUTSU/YOKU", "Port Of Call Online", "willfully ONLINE SHOP","AURALEE"]:
        shopUnit_ex.remove(i)
    df = df.query("ショップ名 in @shopUnit_ex")
    
    if df.empty:
        return
    
    agg = df.groupby(["年", "月", "ショップ名", "申込種別"]).agg({'顧客コード':'nunique'})
    agg.rename(columns={'顧客コード':'申込件数'}, inplace=True)
    
    agg = agg.stack().reset_index()
    
    agg["属性"] = agg["level_4"] + "/" + agg["申込種別"]
    
    return agg

def brand_regMember(df):
    brandUnit_ex = brandUnit.copy()
    for i in ["Ailand", "SANKO BAZAAR", "URBAN RESEARCH OUTLET"]:
        brandUnit_ex.remove(i)
    df = df.query("ショップ名 in @brandUnit_ex")
    
    if df.empty:
        return
    
    agg = df.groupby(["年", "月", "ショップ名", "申込種別"]).agg({'顧客コード':'nunique'})
    agg.rename(columns={'顧客コード':'申込件数'}, inplace=True)
    
    agg = agg.stack().reset_index()
    
    agg["属性"] = agg["level_4"] + "/" + agg["申込種別"]
    
    df_atomic_notNull = df.query("ショップ名 == 'atomicboxx' & 初回受注日 != '< NULL >' ")
    agg_atomic = df_atomic_notNull.groupby(["年", "月", "ショップ名", "申込種別"]).agg({'顧客コード':'nunique'})
    agg_atomic.rename(columns={'顧客コード':'初回購入者'}, inplace=True)
    
    agg_atomic = agg_atomic.stack().reset_index()
    
    agg_atomic["属性"] = agg_atomic["level_4"] + "/" + agg_atomic["申込種別"]
    
    agg_total = pd.concat([agg, agg_atomic], axis = 0)

    return agg_total

def total_regMember(df):
    df = df.replace(regMember_convert)
    df = df.query("ショップ名 in @regMemberTotalUnit")
    
    if df.empty:
        return
    
    agg = df.groupby(["年", "月", "ショップ名", "申込種別"]).agg({'正会員数':'sum'})
    
    agg = agg.stack().reset_index()
    
    agg["属性"] = "会員累計/" + agg["申込種別"]
    
    return agg


In [28]:
#regMember処理
files_regMember = file_ditect("regMember")
files_regMember_total = file_ditect("regMember/" + ym + "/total")

/Users/miyamoto/Documents/Root/raw/monthly_agg/regMember/202101/正会員数獲得データ_日付絞込_20210202022517_shibuya.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/regMember/202101/正会員数獲得データ_日付絞込_20210202022612_AP.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/regMember/202101/正会員数獲得データ_日付絞込_20210202022405_CAN.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/regMember/202101/正会員数獲得データ_日付絞込_20210202022020_new.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/regMember/202101/正会員数獲得データ_日付絞込_20210202022135_stripe.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/regMember/202101/正会員数獲得データ_日付絞込_20210202022711_TOW.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/regMember/202101/正会員数獲得データ_日付絞込_20210202022256_zero.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/regMember/202101/total/202101/正会員数獲得データ_累積合計_20210202022650_TOW.csv
/Users/miyamoto/Documents/Root/raw/monthly_agg/regMember/202101/total/202101/正会員数獲得データ_累積合計_20210202022231_zero.csv
/Users/miyamoto/Documents/Root/raw/mont

In [29]:
regMember_list_shop = []
regMember_list_brand = []
regMember_list_total = []

for file in files_regMember:
    regMember = import_regMember(file)
    
    df_shop_regMember = shop_regMember(regMember)
    df_brand_regMember = brand_regMember(regMember)

    regMember_list_shop.append(df_shop_regMember)
    regMember_list_brand.append(df_brand_regMember)
    
for file in files_regMember_total:
    regMember_total = import_regMember_total(file)

    df_total_regMember = total_regMember(regMember_total)

    regMember_list_shop.append(df_total_regMember)
    regMember_list_brand.append(df_total_regMember)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [30]:
sh_regMember = pd.concat(regMember_list_shop,axis = 0).reset_index()
sh_regMember = sh_regMember[sh_regMember.columns[sh_regMember.columns != 'index']]
sh_regMember.rename(columns={0:"scalar"}, inplace =True)
sh_regMember = sh_regMember[["年", "月", "ショップ名", "属性", "scalar"]]

br_regMember = pd.concat(regMember_list_brand,axis = 0).reset_index()
br_regMember = br_regMember[br_regMember.columns[br_regMember.columns != 'index']]
br_regMember.rename(columns={0:"scalar"}, inplace =True)
br_regMember = br_regMember[["年", "月", "ショップ名", "属性", "scalar"]]

In [35]:
sh_total = pd.concat([sh_sell, sh_order, sh_request, sh_newProduct, sh_receive, sh_regMember], axis = 0)
br_total = pd.concat([br_sell, br_order, br_newProduct, br_request, br_receive, br_regMember], axis = 0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [36]:
sh_total['年'] = sh_total['年'].astype(int)
sh_total['月'] = sh_total['月'].astype(int)
sh_total = sh_total[["年", "月", "ショップ名","属性", "scalar"]].reset_index()
sh_total = sh_total[sh_total.columns[sh_total.columns != 'index']]

br_total['年'] = br_total['年'].astype(int)
br_total['月'] = br_total['月'].astype(int)
br_total = br_total[["年", "月", "ショップ名", "ブランド名", "属性", "scalar"]].reset_index()
br_total = br_total[br_total.columns[br_total.columns != 'index']]

In [37]:
sh_total

,年,月,ショップ名,属性,scalar
0,2021,1,ECLIN,出荷金額,1650789.0
1,2021,1,ECLIN,返品金額,74996.0
2,2021,1,ECLIN,売上金額,1575793.0
3,2021,1,ENVYM,出荷金額,3076340.0
4,2021,1,ENVYM,返品金額,399600.0
5,2021,1,ENVYM,売上金額,2676740.0
6,2021,1,公式通販Fig＆Viper,出荷金額,1903452.0
7,2021,1,公式通販Fig＆Viper,返品金額,139000.0
8,2021,1,公式通販Fig＆Viper,売上金額,1764452.0
9,2021,1,TOKYO OUTLET WEEK ONLINE,出荷金額,131058.0


In [38]:
for shopDf in shopUnit:
    csv_shop = sh_total.query("ショップ名 == @shopDf")
    csv_shop["ショップ名"] = csv_shop["ショップ名"] + "_サイト"
    csv_shop = csv_shop[["年", "月", "ショップ名", "属性", "scalar"]]
    csv_shop.to_csv('./monthly/' + shopDf + '_月次レビュー.csv', encoding='utf_8_sig')
    
for brandDf in brandUnit:
    csv_brand = br_total.query("ショップ名 == @brandDf")
    if brandDf == "Ailand" or brandDf == "URBAN RESEARCH OUTLET":
        brand_inShop = sh_total.query("ショップ名 == @brandDf")
        csv_brand = pd.concat([csv_brand, brand_inShop], axis = 0)
    csv_brand["ショップ名"] = csv_brand["ショップ名"] + "_サイト"
    csv_brand = csv_brand[["年", "月", "ショップ名", "ブランド名", "属性", "scalar"]]
    csv_brand.to_csv('./monthly/' + brandDf + '_月次レビュー.csv', encoding='utf_8_sig')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to no